In [ ]:
from src.learner import *
import time
import pandas as pd
from utilities import remove_cols

# the words were subsets of the Harm set
words = pd.read_csv('data/harm/labels.csv', header=None)
words = words[0].tolist()

# inputs and outputs
X = remove_cols(np.genfromtxt('data/harm/orth.csv', delimiter=","))
Y = remove_cols(np.genfromtxt('data/harm/phon.csv', delimiter=","))

In [ ]:
easy_900 = pd.read_csv('data/harm/easy_900.csv', header=None)[0].tolist()
sample = np.full(len(words), False, dtype=bool)

indices = [i for i, e in enumerate(words) if e in easy_900]
# Set chosen indices to True because they select the test items not the train items
sample[indices] = True

In [ ]:
model = learner(X, Y, seed=seed, hidden=100, optimizer=Adam(learning_rate=.075))    

start_time = time.time()


model.fit(X[sample], Y[sample], epochs=50, batch_size=256, verbose=True)

end_time = time.time()
runtime = end_time - start_time

In [ ]:
seed = 387

with open('outputs/tune_1_harm_2.csv', 'w') as f:
    f.write("{},{},{},{},{},{},{},{},{},{},{}\n".format(
                                            "hidden_units",
                                            "learning_rate",
                                             "batch_size",
                                             "epochs",
                                             "loss_train",
                                             "accuracy_train",
                                             "mse_train",
                                             "loss_test",
                                             "accuracy_test",
                                             "mse_test",
                                             "time"))
    for learning_rate in [.1, .2, .3, .4, .5, .6, .7, .8, .9]: 
        for batch_size in [256, 384, 512]:
            for epochs in [50, 100, 150, 250, 500, 100]:
                for hidden in [100, 150, 200]:

                    model = learner(X, Y, seed=seed, hidden=hidden, optimizer=Adam(learning_rate=learning_rate))    
                    
                    start_time = time.time()


                    model.fit(X, Y, epochs=epochs, batch_size=batch_size, verbose=False)

                    end_time = time.time()
                    runtime = end_time - start_time

                    loss_train, accuracy_train, mse_train = model.evaluate(X, Y, verbose=0) 
                    loss_test, accuracy_test, mse_test = model.evaluate(X, Y, verbose=0) 

                    f.write("{},{},{},{},{},{},{},{},{},{},{}\n".format(
                                                    hidden,
                                                    learning_rate,
                                                    batch_size,
                                                    epochs,
                                                    loss_train,
                                                    accuracy_train,
                                                    mse_train,
                                                    loss_test,
                                                    accuracy_test,
                                                    mse_test,
                                                    runtime))
f.close()